# Add games to analytics directory as json

## Setup

In [550]:
import os
from os import listdir
from os.path import isfile, join
import re
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt

my_id = "aPmT3TcxYLaUAlxSDKDsfKFmXC72"

## Add Single Game 

In [417]:
def write_game_json(game_path):
    command = 'node stats.js'
    os.system(command + ' ' + game_path)

# write_game_json("test_game.slp")

## Add multiple games


In [423]:
def write_games_json(games_path):
    game_files = [games_path + '/' + f for f in listdir(games_path) if isfile(join(games_path, f))]
    # this took about an hour for 6000 games
    for file_name in game_files:
        if not os.path.exists(file_name):
            write_game_json(file_name)

In [424]:
write_games_json("../../Slippi")

# Import game analytics into Pandas

## Single game

In [573]:
def load_single_game(game_file, games):
    
    with open(game_file) as f:
        d = json.load(f)
    
    with open("characters.json") as f:
        characters = json.load(f)
    
    with open("moves.json") as f:
        moves = json.load(f)
    
    with open("stages.json") as f:
        stages = json.load(f)
    
    
    df = pd.json_normalize(d, max_level=1)
    if d['metadata']:
        x = pd.DataFrame()
        x['game_length'] = df['stats.playableFrameCount'] / 60
        x['stage'] = df['settings.stageId'].astype(str).map(stages)
        
        player_one = df["settings.players"][0][0]
        player_two = df["settings.players"][0][1]
        x['player_one_display_name'] = player_one["displayName"]
        x['player_two_display_name'] = player_two["displayName"]
        x['player_one_connect_code'] = player_one["connectCode"]
        x['player_two_connect_code'] = player_two["connectCode"]
        x['player_one_user_id'] = player_one["userId"]
        x['player_two_user_id'] = player_two["userId"]
    
        x['player_one_character'] = characters[str(player_one["characterId"])]['name']
        x['player_two_character'] = characters[str(player_two["characterId"])]['name']
    
        # These won't work till default colors are added (prepended) to characters.json
        # x['player_one_character_color'] = characters[str(player_one["characterId"])]['colors'][player_one["characterColor"]]
        # x['player_two_character_color'] = characters[str(player_two["characterId"])]['colors'][player_two["characterColor"]]
    
        stock_lost_counter = [0, 0]
        stocks_lost = [stock['playerIndex'] for stock in df['stats.stocks'][0] if stock['endFrame']]
        for stock in stocks_lost:
            if stock == 0:
                stock_lost_counter[0] += 1
            else:
                stock_lost_counter[1] += 1
    
        x['player_one_lost_stocks'] = stock_lost_counter[0]
        x['player_two_lost_stocks'] = stock_lost_counter[1]
        if stock_lost_counter[0] == stock_lost_counter[1]:
            x['winner'] = None
            x['winner_id'] = None
        elif stock_lost_counter[0] < stock_lost_counter[1]:
            x['winner'] = 0
            x['winner_id'] = player_one["userId"]
            x['winning_character'] = x["player_one_character"]
            x['loser_id'] = player_two["userId"]
            x['losing_character'] = x["player_two_character"]

        else:
            x['winner'] = 1
            x['winner_id'] = player_two["userId"]
            x['winning_character'] = x["player_two_character"]
            x['loser_id'] = player_one["userId"]
            x['losing_character'] = x["player_one_character"]

        x['played_at'] = df['metadata.startAt']
        x['overall'] = df['stats.overall']
        x['action_counts'] = df['stats.actionCounts']
        x['game_complete'] = df['stats.gameComplete']


        # movement
        x['player_one_failed_l_cancels'] = df['stats.actionCounts'][0][0]['lCancelCount']['fail']
        x['player_one_successful_l_cancels'] = df['stats.actionCounts'][0][0]['lCancelCount']['success']
        x['player_two_failed_l_cancels'] = df['stats.actionCounts'][0][1]['lCancelCount']['fail']
        x['player_two_successful_l_cancels'] = df['stats.actionCounts'][0][1]['lCancelCount']['success']

        x['player_one_wavedash_count'] = df['stats.actionCounts'][0][0]['wavedashCount']
        x['player_one_waveland_count'] = df['stats.actionCounts'][0][0]['wavelandCount']
        x['player_two_wavedash_count'] = df['stats.actionCounts'][0][1]['wavedashCount']
        x['player_two_waveland_count'] = df['stats.actionCounts'][0][1]['wavelandCount']

                
        # airDodgeCount
        # dashDanceCount
        # spotDodgeCount
        # ledgegrabCount
        # rollCount

        # attackCount
        # throwCount
        # grabCount
        # wallTechCount

        # groundTechCount
        x['player_one_tech_away'] = df['stats.actionCounts'][0][0]['groundTechCount']['away']
        x['player_one_tech_in'] = df['stats.actionCounts'][0][0]['groundTechCount']['in']
        x['player_one_tech_neutral'] = df['stats.actionCounts'][0][0]['groundTechCount']['neutral']
        x['player_one_tech_fail'] = df['stats.actionCounts'][0][0]['groundTechCount']['fail']

        x['player_two_tech_away'] = df['stats.actionCounts'][0][1]['groundTechCount']['away']
        x['player_two_tech_in'] = df['stats.actionCounts'][0][1]['groundTechCount']['in']
        x['player_two_tech_neutral'] = df['stats.actionCounts'][0][1]['groundTechCount']['neutral']
        x['player_two_tech_fail'] = df['stats.actionCounts'][0][1]['groundTechCount']['fail']

        
        games = pd.concat([games, x], ignore_index=True)
        # print(games.shape)
        return games
    else:
        print("something went wrong with: ", game_file)
        # print(games.shape)
        return games


# games = load_single_game('game_analytics/Game_20230809T191315.json', pd.DataFrame())
# games

## Multiple games

In [574]:
def load_multiple_games(analytics_path):

    analytics_files = [analytics_path + '/' + f for f in listdir(analytics_path) if isfile(join(analytics_path, f))]
    
    games = pd.DataFrame()
    for i, f in enumerate(analytics_files):
        # print(i)
        # if i>1000:
        #     break
        try:
            games = load_single_game(f, games)
        except:
            print("An exception occurred for: ") 
            print(f)
    return games

all_games = load_multiple_games("game_analytics")

something went wrong with:  game_analytics/Game_20231002T112717.json
something went wrong with:  game_analytics/Game_20230821T074359.json
something went wrong with:  game_analytics/.DS_Store
something went wrong with:  game_analytics/Game_20231003T144755.json
something went wrong with:  game_analytics/Game_20230810T141105.json
something went wrong with:  game_analytics/Game_20231001T235749.json
something went wrong with:  game_analytics/Game_20231018T195855.json
something went wrong with:  game_analytics/Game_20230827T101530.json
something went wrong with:  game_analytics/Game_20230807T091818.json
something went wrong with:  game_analytics/Game_20230806T200651.json
something went wrong with:  game_analytics/Game_20230814T213419.json
something went wrong with:  game_analytics/Game_20230722T085734.json
something went wrong with:  game_analytics/Game_20231021T081930.json
something went wrong with:  game_analytics/Game_20230906T150537.json
something went wrong with:  game_analytics/Game_20

In [576]:
all_games



,game_length,stage,player_one_display_name,player_two_display_name,player_one_connect_code,player_two_connect_code,player_one_user_id,player_two_user_id,player_one_character,player_two_character,...,player_one_tech_in,player_one_tech_neutral,player_one_tech_fail,player_two_tech_away,player_two_tech_in,player_two_tech_neutral,player_two_tech_fail,winning_character,loser_id,losing_character
0,32.900000,Pokémon Stadium,with,meow,WITH#0,MEOW#339,aPmT3TcxYLaUAlxSDKDsfKFmXC72,mwKYA5rvxaYBjYNpdEe8b39L7un2,Yoshi,Marth,...,0,0,1,0,0,0,0,NaN,NaN,NaN
1,217.083333,Yoshi's Story,with,VHSTAPE,WITH#0,VHS#396,aPmT3TcxYLaUAlxSDKDsfKFmXC72,WweIVWmBLXSa9Tv6VG8dmYPDJ8Z2,Yoshi,Peach,...,0,0,4,0,0,1,2,Peach,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Yoshi
2,102.533333,Pokémon Stadium,Matcha Man,with,MATC#818,WITH#0,Es93fbpkTRMBc1EPfiSHlfs2cMv1,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Peach,Yoshi,...,0,0,0,0,0,1,2,Yoshi,Es93fbpkTRMBc1EPfiSHlfs2cMv1,Peach
3,46.583333,Yoshi's Story,with,Shlomp,WITH#0,CLOR#884,aPmT3TcxYLaUAlxSDKDsfKFmXC72,30cOiOMWLraUtgA8JltYkeO3cI82,Yoshi,Marth,...,0,0,0,1,0,0,0,NaN,NaN,NaN
4,65.166667,Fountain of Dreams,ITSU,with,ITSU#117,WITH#0,NwRhZmeQYhQL13p8BCGqSLYGwAG2,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Dr. Mario,Yoshi,...,0,0,2,0,0,0,3,Yoshi,NwRhZmeQYhQL13p8BCGqSLYGwAG2,Dr. Mario
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6055,228.950000,Yoshi's Story,with,Jotyde,WITH#0,JOTY#425,aPmT3TcxYLaUAlxSDKDsfKFmXC72,OZhNnyBDstWxqxN8Sic9YPsns9B3,Yoshi,Fox,...,0,0,7,1,0,3,9,Yoshi,OZhNnyBDstWxqxN8Sic9YPsns9B3,Fox
6056,36.716667,Pokémon Stadium,with,bradday0429,WITH#0,BRAD#102,aPmT3TcxYLaUAlxSDKDsfKFmXC72,sVNWjJMzpGQ4w4iZFxcp0nvnL3n2,Yoshi,Luigi,...,0,0,2,0,1,0,1,Yoshi,sVNWjJMzpGQ4w4iZFxcp0nvnL3n2,Luigi
6057,157.400000,Fountain of Dreams,KEV,with,SSC#23,WITH#0,7eIllMJ4nCSpZnZwoYJCJMfKRnq2,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Captain Falcon,Yoshi,...,1,1,3,0,1,1,2,Yoshi,7eIllMJ4nCSpZnZwoYJCJMfKRnq2,Captain Falcon
6058,239.016667,Dream Land N64,with,ttocs,WITH#0,TTOX#608,aPmT3TcxYLaUAlxSDKDsfKFmXC72,W7JRcoIdRyMSCXHquOMoB8EOT7L2,Yoshi,Captain Falcon,...,0,0,4,1,0,2,7,Captain Falcon,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Yoshi


# Queries

## Win Rates

In [483]:
def finished_games(games, my_id):
    return games.loc[(games['winner_id'].notnull())]
    
def wins(games, my_id):
    return games.loc[(games['winner_id'] == my_id)]

def losses(games, my_id):
    return games.loc[(games['loser_id'] == my_id)]
    
def overall_win_rates(games, my_id):
    w = wins(games, my_id)
    l = losses(games, my_id)
    undetermined = games.loc[(games['winner_id'].isnull())]
    if w.shape[0] + l.shape[0] == 0:
        return w.shape[0], l.shape[0], 0
    return w.shape[0], l.shape[0], w.shape[0] / (w.shape[0] + l.shape[0])


def win_rates_by_character(games, my_id, my_character, their_character):
    
    w = wins(games, my_id)
    l = losses(games, my_id)
    
    w = w.loc[(w['winning_character'] == my_character) & (w['losing_character'] == their_character)]
    l = l.loc[(l['winning_character'] == their_character) & (l['losing_character'] == my_character)]
    if w.shape[0] + l.shape[0] == 0:
        return w.shape[0], l.shape[0], 0
    return w.shape[0], l.shape[0], w.shape[0] / (w.shape[0] + l.shape[0])

def win_rates_by_stage(games, my_id, stage):
    w = wins(games, my_id)
    l = losses(games, my_id)
    
    w = w.loc[(w['stage'] == stage)]
    l = l.loc[(l['stage'] == stage)]
    if w.shape[0] + l.shape[0] == 0:
        return w.shape[0], l.shape[0], 0
    return w.shape[0], l.shape[0], w.shape[0] / (w.shape[0] + l.shape[0])

def win_rates_for_all_characters(games, my_id):
        
    char_names = ["Captain Falcon", "Donkey Kong", "Fox", "Mr. Game & Watch", "Kirby", "Bowser",
     "Link", "Luigi", "Mario", "Marth", "Mewtwo", "Ness", "Peach", "Pikachu", "Ice Climbers", 
     "Jigglypuff", "Samus", "Yoshi", "Zelda", "Sheik", "Falco", "Young Link", "Dr. Mario", "Roy", "Pichu", "Ganondorf"]
    
    win_rates = {}
    for char in char_names:
        char_wins = win_rates_by_character(games, my_id, "Yoshi", char)
        win_rates[char] = char_wins[2]
        # print("\nwin rate vs " + char + ":\n", char_wins[0], "\nlosses: ", char_wins[1], "\nwin rate: ", char_wins[2])
    return win_rates

def print_overall_win_rates(games, my_id):
    overall = overall_win_rates(games, my_id)
    print("wins: ", overall[0], "\nlosses: ", overall[1], "\nwin rate: ", overall[2])

def win_rates_for_all_stages(games, my_id):

    stages = ["Fountain of Dreams",
            "Pokémon Stadium",
            "Princess Peach's Castle",
            "Kongo Jungle",
            "Brinstar",
            "Corneria",
            "Yoshi's Story",
            "Onett",
            "Mute City",
            "Rainbow Cruise",
            "Jungle Japes",
            "Great Bay",
            "Hyrule Temple",
            "Brinstar Depths",
            "Yoshi's Island",
            "Green Greens",
            "Fourside",
            "Mushroom Kingdom I",
            "Mushroom Kingdom II"
            "Venom",
            "Poké Floats",
            "Big Blue",
            "Icicle Mountain",
            "Icetop",
            "Flat Zone",
            "Dream Land N64",
            "Yoshi's Island N64",
            "Kongo Jungle N64",
            "Battlefield",
            "Final Destination"]
    win_rates = {}
    for stage in stages:
        stage_wins = win_rates_by_stage(games, my_id, stage)
        win_rates[stage] = stage_wins[2]
        # print("\nwin rate vs " + char + ":\n", char_wins[0], "\nlosses: ", char_wins[1], "\nwin rate: ", char_wins[2])
    win_rates = {k: v for k, v in sorted(win_rates.items(), key=lambda item: item[1], reverse=True)}
    return win_rates

win_rates_for_all_stages(all_games, my_id)

{'Dream Land N64': 0.5213675213675214,
 'Final Destination': 0.4827586206896552,
 "Yoshi's Story": 0.4734982332155477,
 'Pokémon Stadium': 0.4641638225255973,
 'Fountain of Dreams': 0.451348182883939,
 'Battlefield': 0.4310747663551402,
 "Princess Peach's Castle": 0,
 'Kongo Jungle': 0,
 'Brinstar': 0,
 'Corneria': 0,
 'Onett': 0,
 'Mute City': 0,
 'Rainbow Cruise': 0,
 'Jungle Japes': 0,
 'Great Bay': 0,
 'Hyrule Temple': 0,
 'Brinstar Depths': 0,
 "Yoshi's Island": 0,
 'Green Greens': 0,
 'Fourside': 0,
 'Mushroom Kingdom I': 0,
 'Mushroom Kingdom IIVenom': 0,
 'Poké Floats': 0,
 'Big Blue': 0,
 'Icicle Mountain': 0,
 'Icetop': 0,
 'Flat Zone': 0,
 "Yoshi's Island N64": 0,
 'Kongo Jungle N64': 0}

In [484]:
def get_character_win_rates(games, my_id):
    win_rates = win_rates_for_all_characters(games, my_id)
    return {k: v for k, v in sorted(win_rates.items(), key=lambda item: item[1])}

get_character_win_rates(all_games, my_id)

{'Sheik': 0.3032069970845481,
 'Pikachu': 0.379746835443038,
 'Zelda': 0.38461538461538464,
 'Captain Falcon': 0.3883495145631068,
 'Peach': 0.4152542372881356,
 'Samus': 0.42452830188679247,
 'Ganondorf': 0.4420289855072464,
 'Mr. Game & Watch': 0.4444444444444444,
 'Yoshi': 0.45806451612903226,
 'Fox': 0.4648786717752235,
 'Falco': 0.4661582459485224,
 'Marth': 0.4773462783171521,
 'Link': 0.4827586206896552,
 'Dr. Mario': 0.4876543209876543,
 'Luigi': 0.4896551724137931,
 'Donkey Kong': 0.6052631578947368,
 'Ice Climbers': 0.6075949367088608,
 'Ness': 0.6190476190476191,
 'Bowser': 0.6363636363636364,
 'Roy': 0.6388888888888888,
 'Jigglypuff': 0.649746192893401,
 'Mario': 0.6704545454545454,
 'Young Link': 0.6833333333333333,
 'Kirby': 0.7777777777777778,
 'Pichu': 0.7857142857142857,
 'Mewtwo': 0.8378378378378378}

In [485]:
connect_code_pattern = re.compile("^[A-Z]+\#[0-9]+$")
id_pattern = re.compile("^([A-Z]|[a-z]|[0-9]){28}$")

def get_opponent_history(games, opponent):
    if id_pattern.match(opponent):
        return get_opponent_history_by_id(games, opponent)    
    elif connect_code_pattern.match(opponent):
        return get_opponent_history_by_connect_code(games, opponent)    
    else:
        return get_opponent_history_by_display_name(games, opponent)    

def get_opponent_history_by_id(games, opponent):
    return games.loc[(games["player_one_user_id"] == opponent) | (games["player_two_user_id"] == opponent)] 
def get_opponent_history_by_connect_code(games, opponent):
    return games.loc[(games["player_one_connect_code"] == opponent) | (games["player_two_connect_code"] == opponent)] 
def get_opponent_history_by_display_name(games, opponent):
    return games.loc[(games["player_one_display_name"] == opponent) | (games["player_two_display_name"] == opponent)] 


In [409]:
games = get_opponent_history(all_games, "Kappa")
games

,game_length,stage,player_one_display_name,player_two_display_name,player_one_connect_code,player_two_connect_code,player_one_user_id,player_two_user_id,player_one_character,player_two_character,...,player_one_successful_l_cancels,player_two_failed_l_cancels,player_two_successful_l_cancels,player_one_wavedash_count,player_one_waveland_count,player_two_wavedash_count,player_two_waveland_count,winning_character,loser_id,losing_character
9,103.733333,Dream Land N64,with,Kappa,WITH#0,KAPP#698,aPmT3TcxYLaUAlxSDKDsfKFmXC72,A4xiKH9z62cgsIonvehJqMhTyn73,Yoshi,Fox,...,0,5,22,0,0,14,5,Fox,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Yoshi
19,130.933333,Final Destination,with,Kappa,WITH#0,KAPP#698,aPmT3TcxYLaUAlxSDKDsfKFmXC72,A4xiKH9z62cgsIonvehJqMhTyn73,Yoshi,Captain Falcon,...,0,2,26,0,0,5,3,Captain Falcon,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Yoshi
52,122.083333,Battlefield,Kappa,with,KAPP#698,WITH#0,A4xiKH9z62cgsIonvehJqMhTyn73,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Sheik,Yoshi,...,3,7,0,7,1,0,0,Sheik,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Yoshi
60,141.083333,NaN,Kappa,with,KAPP#698,WITH#0,A4xiKH9z62cgsIonvehJqMhTyn73,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Fox,Yoshi,...,24,12,0,23,3,0,0,Fox,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Yoshi
76,144.133333,Dream Land N64,with,Kappa,WITH#0,KAPP#698,aPmT3TcxYLaUAlxSDKDsfKFmXC72,A4xiKH9z62cgsIonvehJqMhTyn73,Yoshi,Sheik,...,0,2,7,0,0,6,2,Sheik,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Yoshi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5859,309.916667,Battlefield,Kappa,with,KAPP#698,WITH#0,A4xiKH9z62cgsIonvehJqMhTyn73,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Ice Climbers,Ice Climbers,...,26,1,0,171,14,0,0,Ice Climbers,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Ice Climbers
5861,77.100000,NaN,Kappa,with,KAPP#698,WITH#0,A4xiKH9z62cgsIonvehJqMhTyn73,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Fox,Ice Climbers,...,17,5,0,22,4,0,0,Fox,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Ice Climbers
5881,7.716667,Yoshi's Story,with,Kappa,WITH#0,KAPP#698,aPmT3TcxYLaUAlxSDKDsfKFmXC72,A4xiKH9z62cgsIonvehJqMhTyn73,Yoshi,Captain Falcon,...,0,0,1,0,0,0,0,NaN,NaN,NaN
5898,96.500000,Yoshi's Story,Kappa,with,KAPP#698,WITH#0,A4xiKH9z62cgsIonvehJqMhTyn73,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Fox,Yoshi,...,14,13,0,29,1,0,0,Fox,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Yoshi


## Encounter rates

In [457]:
def encounter_rates_by_character(games, my_id, their_character):
    a = games.loc[(games['winner_id'] == my_id) & (games['losing_character'] == their_character)]
    b = games.loc[(games['loser_id'] == my_id) & (games['winning_character'] == their_character)]
    c = pd.concat([a, b], ignore_index=True)
    wr = overall_win_rates(games, my_id)

    return c.shape[0] / (wr[0] + wr[1])

def encounter_rates_across_characters(games, my_id):
    char_names = ["Captain Falcon", "Donkey Kong", "Fox", "Mr. Game & Watch", "Kirby", "Bowser",
     "Link", "Luigi", "Mario", "Marth", "Mewtwo", "Ness", "Peach", "Pikachu", "Ice Climbers", 
     "Jigglypuff", "Samus", "Yoshi", "Zelda", "Sheik", "Falco", "Young Link", "Dr. Mario", "Roy", "Pichu", "Ganondorf"]
    
    encounter_rates = {}
    for char in char_names:
        char_encounters = encounter_rates_by_character(games, my_id, char)
        encounter_rates[char] = char_encounters
    return {k: v for k, v in sorted(encounter_rates.items(), key=lambda item: item[1], reverse=True)}

encounter_rates_across_characters(all_games, my_id)

{'Falco': 0.2056420233463035,
 'Fox': 0.15369649805447472,
 'Captain Falcon': 0.1208171206225681,
 'Marth': 0.12062256809338522,
 'Sheik': 0.06750972762645914,
 'Jigglypuff': 0.03832684824902724,
 'Dr. Mario': 0.03151750972762646,
 'Yoshi': 0.030155642023346304,
 'Luigi': 0.02821011673151751,
 'Ganondorf': 0.027431906614785992,
 'Peach': 0.022957198443579768,
 'Samus': 0.020622568093385214,
 'Mario': 0.017120622568093387,
 'Ice Climbers': 0.015953307392996108,
 'Pikachu': 0.01536964980544747,
 'Donkey Kong': 0.014785992217898832,
 'Roy': 0.014007782101167316,
 'Young Link': 0.011673151750972763,
 'Link': 0.011284046692607004,
 'Mewtwo': 0.0071984435797665365,
 'Mr. Game & Watch': 0.007003891050583658,
 'Bowser': 0.004280155642023347,
 'Ness': 0.004085603112840467,
 'Kirby': 0.003501945525291829,
 'Pichu': 0.002723735408560311,
 'Zelda': 0.002529182879377432}

## Movement

In [571]:
def l_cancel_rate(games, my_id, me=True):
    games = finished_games(games, my_id)

    if me:
        p1 = games.loc[(games['player_one_user_id'] == my_id)][['player_one_successful_l_cancels', 'player_one_failed_l_cancels']]
        p2 = games.loc[(games['player_two_user_id'] == my_id)][['player_two_successful_l_cancels', 'player_two_failed_l_cancels']]
    else:
        p1 = games.loc[(games['player_two_user_id'] == my_id)][['player_one_successful_l_cancels', 'player_one_failed_l_cancels']]
        p2 = games.loc[(games['player_one_user_id'] == my_id)][['player_two_successful_l_cancels', 'player_two_failed_l_cancels']]
    
    successful = p1[['player_one_successful_l_cancels']].sum().iloc[0] + p2[['player_two_successful_l_cancels']].sum().iloc[0]
    failed = p1[['player_one_failed_l_cancels']].sum().iloc[0] + p2[['player_two_failed_l_cancels']].sum().iloc[0]
    return successful / (successful + failed)

def l_cancel_ratio_over_time(games, my_id, batches, me=True):
    games = finished_games(games, my_id)

    if me:
        p1 = games.loc[(games['player_one_user_id'] == my_id)][['player_one_successful_l_cancels', 'player_one_failed_l_cancels']].rename(columns = {'player_one_successful_l_cancels':'success', 'player_one_failed_l_cancels':'failed'})
        p2 = games.loc[(games['player_two_user_id'] == my_id)][['player_two_successful_l_cancels', 'player_two_failed_l_cancels']].rename(columns = {'player_two_successful_l_cancels':'success', 'player_two_failed_l_cancels':'failed'})
    else:
        p1 = games.loc[(games['player_one_user_id'] == my_id)][['player_two_successful_l_cancels', 'player_two_failed_l_cancels']].rename(columns = {'player_two_successful_l_cancels':'success', 'player_two_failed_l_cancels':'failed'})
        p2 = games.loc[(games['player_two_user_id'] == my_id)][['player_one_successful_l_cancels', 'player_one_failed_l_cancels']].rename(columns = {'player_one_successful_l_cancels':'success', 'player_one_failed_l_cancels':'failed'})
        
    p = pd.concat([p1, p2]).sort_index().reset_index(drop=True)
    p['batch'] = (p.index) // (p.shape[0] // batches)
    # print(p)

    g1 = p.groupby('batch')['success'].sum()
    g2 = p.groupby('batch')['failed'].sum()

    return (g1 / (g1 + g2))

def show_l_cancel_rate_over_time(games, my_id, batches):
    x = np.linspace(0, 1, batches+1)
    y = l_cancel_rate_over_time(games, my_id, batches)    
    fig, ax = plt.subplots()
    ax.plot(x, y)
    plt.show()

l_cancel_rate(finished_games(games, my_id), my_id, False)

0.5443554606394969

In [594]:
def tech_options(games, my_id, me=True):
    games = finished_games(games, my_id)

    if me:
        p1 = games.loc[(games['player_one_user_id'] == my_id)][['player_one_character', 'player_one_tech_away', 'player_one_tech_in', 'player_one_tech_neutral', 'player_one_tech_fail']]
        p2 = games.loc[(games['player_two_user_id'] == my_id)][['player_two_character', 'player_two_tech_away', 'player_two_tech_in', 'player_two_tech_neutral', 'player_two_tech_fail']]
    else:
        p1 = games.loc[(games['player_two_user_id'] == my_id)][['player_one_character', 'player_one_tech_away', 'player_one_tech_in', 'player_one_tech_neutral', 'player_one_tech_fail']]
        p2 = games.loc[(games['player_one_user_id'] == my_id)][['player_two_character', 'player_two_tech_away', 'player_two_tech_in', 'player_two_tech_neutral', 'player_two_tech_fail']]
    
    p1.rename(columns = {'player_one_tech_away': 'tech_away', 
                         'player_one_tech_in': 'tech_in',
                         'player_one_tech_neutral': 'tech_neutral',
                         'player_one_tech_fail': 'tech_fail',
                         'player_one_character': 'character'}, inplace=True)

    p2.rename(columns = {'player_two_tech_away': 'tech_away', 
                         'player_two_tech_in': 'tech_in',
                         'player_two_tech_neutral': 'tech_neutral',
                         'player_two_tech_fail': 'tech_fail',
                         'player_two_character': 'character'}, inplace=True)

    p = pd.concat([p1, p2]).sort_index().reset_index(drop=True)
    return p

tech_options_for_me = tech_options(all_games, my_id)
totals = tech_options_for_me[['tech_away', 'tech_in', 'tech_fail', 'tech_neutral']].sum()
total = totals.sum()
print("my tech\n", totals / total)

tech_options_for_thee = tech_options(all_games, my_id, False)
totals = tech_options_for_thee[['tech_away', 'tech_in', 'tech_fail', 'tech_neutral']].sum()
total = totals.sum()
print("\ntheir tech\n", totals / total)




my tech
 tech_away       0.053040
tech_in         0.054487
tech_fail       0.870346
tech_neutral    0.022127
dtype: float64

their tech
 tech_away       0.044778
tech_in         0.060023
tech_fail       0.692127
tech_neutral    0.203072
dtype: float64


In [599]:
def tech_options(games, my_id, character):
    games = finished_games(games, my_id)

    p1 = games.loc[(games['player_two_user_id'] == my_id) & (games['player_one_character'] == character)][['player_one_character', 'player_one_tech_away', 'player_one_tech_in', 'player_one_tech_neutral', 'player_one_tech_fail']]
    p2 = games.loc[(games['player_one_user_id'] == my_id) & (games['player_two_character'] == character)][['player_two_character', 'player_two_tech_away', 'player_two_tech_in', 'player_two_tech_neutral', 'player_two_tech_fail']]
    
    p1.rename(columns = {'player_one_tech_away': 'tech_away', 
                         'player_one_tech_in': 'tech_in',
                         'player_one_tech_neutral': 'tech_neutral',
                         'player_one_tech_fail': 'tech_fail',
                         'player_one_character': 'character'}, inplace=True)

    p2.rename(columns = {'player_two_tech_away': 'tech_away', 
                         'player_two_tech_in': 'tech_in',
                         'player_two_tech_neutral': 'tech_neutral',
                         'player_two_tech_fail': 'tech_fail',
                         'player_two_character': 'character'}, inplace=True)

    p = pd.concat([p1, p2]).sort_index().reset_index(drop=True)
    return p



char_names = ["Captain Falcon", "Donkey Kong", "Fox", "Mr. Game & Watch", "Kirby", "Bowser",
     "Link", "Luigi", "Mario", "Marth", "Mewtwo", "Ness", "Peach", "Pikachu", "Ice Climbers", 
     "Jigglypuff", "Samus", "Yoshi", "Zelda", "Sheik", "Falco", "Young Link", "Dr. Mario", "Roy", "Pichu", "Ganondorf"]
    
tech_stats = {}
for char in char_names:
    tech_options_for_thee = tech_options(all_games, my_id, char)
    totals = tech_options_for_thee[['tech_away', 'tech_in', 'tech_fail', 'tech_neutral']].sum()
    total = totals.sum()
    tech_stats[char] = totals / total

print(tech_stats)



{'Captain Falcon': tech_away       0.061726
tech_in         0.053247
tech_fail       0.654061
tech_neutral    0.230965
dtype: float64, 'Donkey Kong': tech_away       0.050292
tech_in         0.078363
tech_fail       0.651462
tech_neutral    0.219883
dtype: float64, 'Fox': tech_away       0.043385
tech_in         0.047022
tech_fail       0.708165
tech_neutral    0.201428
dtype: float64, 'Mr. Game & Watch': tech_away       0.028571
tech_in         0.020408
tech_fail       0.763265
tech_neutral    0.187755
dtype: float64, 'Kirby': tech_away       0.0375
tech_in         0.0875
tech_fail       0.7125
tech_neutral    0.1625
dtype: float64, 'Bowser': tech_away       0.019231
tech_in         0.070513
tech_fail       0.685897
tech_neutral    0.224359
dtype: float64, 'Link': tech_away       0.036244
tech_in         0.054366
tech_fail       0.742998
tech_neutral    0.166392
dtype: float64, 'Luigi': tech_away       0.032821
tech_in         0.072821
tech_fail       0.706667
tech_neutral    0.187692